# 1 -  Planejamento da Solução

## Entendimento de Negócio

**Qual é o problema de negócio?**

- Empresa alimentícia situada no RJ, deseja abrir filiais na cidade de São Paulo. Para isso, precisa de 3 análises:
    - 1 - Estimar faturamento que uma loja teria em cada um dos bairros de São Paulo (regressão, neste caso sem séries temporais).
    - 2 - Classificar o potencial dos bairros de São Paulo em alto, médio ou baixo (classificação multiclasse).
    - 3 - Segmentar os bairros de São Paulo de acordo com o perfil de renda e idade, identificando os com maior aderência ao público alvo.
    
- Público alvo: adultos de 25 a 50 anos, das classes A (rendas A1 e A2) e B (rendas B1 e B2). 

**Saída**

- Prototipagem técnica da solução: https://docs.google.com/spreadsheets/d/17lxCBRLPEuNCO25BimVFRE3Tms-314WSTpJ1xgEIf38/edit#gid=0

- O que será entregue, efetivamente? / Onde o time de negócio quer ver?
    - Documento no formato doc, pdf ou ppt, voltado ao negócio, apresentando um racional de como os dados foram analisados. Detalhar com gráficos, tabelas, e descrever conclusões.
    - Responder: Dada a natureza do problema apresentado, que outro dado externo (fontes públicas ou privadas) poderia ser utilizado para agregar mais valor ao resultado? Por que?
    
**Entrada**
- Fontes de dados:
    - Dataset contendo faturamento e potencial dos bairros do Rio de Janeiro do cliente, bem como dados sociodemográficos do bairros do Rio de Janeiro e São Paulo.

- Ferramentas:
    - Python 3.8.12, Jupyter Notebook, Git, Github.

**Processamento**
- Tipo de problema: análise exploratória, regressão, classificação e clusterização.
- Metodologia: CRISP-DM, metodologia ágil (iterativa e incremental) para desenvolvimento de projetos de ciência de dados.


## Implementado nesta Sprint

**Ciclo 3**

Desenvolvimento da Análise 2 (classificação):
- Split dos dados.
- Modelagem de dados (data preparation).
- Seleção de Features.
- Model Based feature selection.
- Novo KNN Classifier + comparação com o baseline do ciclo 1.
- KNN Classifier com Random Search + Cross validation.
- Novos algoritmos de ML: Logistic Regression, XGBoost Classifier (embedding de trees).
- Comparação da performance dos modelos.
- Performance de generalização do melhor modelo com dados de teste.
- Modelos final, treinado com dataset traino + validação, prevendo potencial por bairro de SP (produção).

# 2 - Importações

## Bibliotecas

In [505]:
import pandas                      as pd 
import seaborn                     as sns
import numpy                       as np
import sweetviz                    as sv
import inflection
import warnings

from IPython.core.display          import HTML
from matplotlib                    import pyplot as plt
from tabulate                      import tabulate
from matplotlib.ticker             import FuncFormatter
from IPython.display               import Image

from sklearn.model_selection       import train_test_split, GridSearchCV
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.metrics               import accuracy_score
from sklearn.metrics               import f1_score
from sklearn.preprocessing         import MinMaxScaler, OrdinalEncoder
from sklearn.feature_selection     import SelectFromModel, SelectKBest
from sklearn.ensemble              import RandomForestClassifier
from sklearn.linear_model          import LogisticRegression
from xgboost                       import XGBClassifier

## Funções Auxiliares

In [383]:
def jupyter_settings():
    """ Otimiza configurações gerais, padronizando tamanhos de plots, etc """
    %matplotlib inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    pd.set_option('display.max_columns', 30)
    pd.set_option('display.max_rows', 30)
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    pd.set_option('max_colwidth', None)
jupyter_settings()

# 3 - Análise 2: Classificação do potencial em SP

Análise 2 - Classificar o potencial dos bairros de São Paulo em alto, médio ou baixo (classificação multiclasse).

## Split dos Dados

In [533]:
df_rj = pd.read_csv('../data/interim/df_rj_feat_eng_done.csv', index_col=0)
print(df_rj.shape)
df_rj.head(1)

(160, 22)


,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento,potencial
0,11676,1027,483,688,800,1675,2300,1784,2919,3975,0,145,715,1242,1093,758,92,304,2102,2501.0000,932515,Médio


Trata-se portanto de um problema de classificação multiclasse (supervisionado), sendo a variável alvo "potencial" categórica. 
- As possíveis categorias de "potencial" são: Alto, Médio e Baixo.

Assim como na análise 1, aqui df_rj também será dividido em três partes: dados de treinamento, validação e teste. 
- A separação dos dados da análise 1 não será aproveitada, porque a variável alvo era "faturamento", e nesta análise será "potencial".
- Ou seja, nesta análise 2, "faturamento" será uma variável preditora (feature), e "potencial" será a variável dependente (target).

Divisão entre features e variável alvo.

In [534]:
y2 = df_rj.potencial
y2.head(2)

0    Médio
1    Baixo
Name: potencial, dtype: object

A variável "codigo" não é informativa, logo será descartada.

In [535]:
X2 = df_rj.drop("potencial", axis=1)
X2.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
0,11676,1027,483,688,800,1675,2300,1784,2919,3975,0,145,715,1242,1093,758,92,304,2102,2501.0000,932515


Avaliar balanceamento da var resposta.

In [536]:
y2.value_counts(normalize=True)*100

Baixo   38.7500
Médio   31.2500
Alto    30.0000
Name: potencial, dtype: float64

A variável alvo "potencial" está balanceada, ou seja, tem uma distribuição parecida dos registros nas categorias existentes. 
- Isto dispensa tratativas para balanceamento, e permite o uso da métrica de acurácia, para avaliação de performance.

Divisão entre datasets de treino/validação e teste, mantendo 20% dos registros para teste.
- O parâmetro "stratify" é utilizado para manter uma proporção parecida da variável alvo nos dados de treino e de teste.

In [550]:
X2_trainval, X2_test, y2_trainval, y2_test = train_test_split(X2, y2, random_state=0, test_size=0.20, stratify=y2)

Divisão entre datasets de treino e validação (a partir do trainval), mantendo 25% dos registros para validação.

In [551]:
X2_train, X2_val, y2_train, y2_val = train_test_split(X2_trainval, y2_trainval, random_state=0, test_size=0.25, stratify=y2_trainval)

In [552]:
print(X2_train.shape)
print(X2_val.shape)
print(X2_test.shape)
X2_train.head(1)

(96, 21)
(32, 21)
(32, 21)


,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
18,11325,1256,588,817,811,1775,2533,1456,2089,4308,0,71,473,903,1161,712,120,558,1447,2022.0000,852714


## Preparação dos Dados

Tratativas necessárias para as features da análise 2 (classificação).
- São as mesmas da análise 1, com a diferença qua agora a variável resposta é "potencial", logo:
    - Será preciso reescalar também faturamento, que agora é uma feature comum.
- O dataset será ajustado abaixo.

In [553]:
X2_train.columns

Index(['populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24',
       'pop_de25a34', 'pop_de35a49', 'pop_de50a59', 'pop_mais_de60',
       'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1',
       'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d',
       'domicilios_e', 'domicilios_alvo', 'renda_media', 'faturamento'],
      dtype='object')

In [554]:
X2_train.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
18,11325,1256,588,817,811,1775,2533,1456,2089,4308,0,71,473,903,1161,712,120,558,1447,2022.0000,852714


Tratativas necessárias para as features da análise 2 (classificação).
- Numéricas (tratadas com scaling): 'populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24', 'pop_de25a34', 'pop_de35a49', 'pop_de50a59', 'pop_mais_de60', 'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d', 'domicilios_e', 'domicilios_alvo', 'renda_media', 'faturamento'.
- Categóricas (tratadas com encoding): 'potencial' (var resposta).

### Scaling

In [555]:
features_scaling2 = ['populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24', 'pop_de25a34', 'pop_de35a49', 
'pop_de50a59', 'pop_mais_de60', 'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_b2', 
'domicilios_c1', 'domicilios_c2', 'domicilios_d', 'domicilios_e', 'domicilios_alvo', 'renda_media', 'faturamento']

Reescala apenas as features selecionadas do treino com MinMaxScaler.

In [556]:
scaler2 = MinMaxScaler()
X2_train[features_scaling2] = scaler2.fit_transform(X2_train[features_scaling2])
X2_train.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
18,0.0446,0.0375,0.0361,0.0380,0.0394,0.0463,0.0470,0.0463,0.0404,0.0467,0.0000,0.0077,0.0244,0.0680,0.0475,0.0316,0.0263,0.0501,0.0305,0.0804,0.3509


Aplica o scaler já treinado apenas nas features selecionadas de validação.

In [557]:
X2_val[features_scaling2] = scaler2.transform(X2_val[features_scaling2])
X2_val.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
93,0.2044,0.1023,0.1071,0.1305,0.1604,0.1796,0.1850,0.2612,0.2977,0.1827,0.0889,0.2032,0.2823,0.3834,0.1604,0.0698,0.0287,0.1219,0.2725,0.2362,0.6789


Já deixar pronto também trainval, para avaliar a capacidade de generalização no final.

In [558]:
scaler2_tv = MinMaxScaler()
X2_trainval[features_scaling2] = scaler2_tv.fit_transform(X2_trainval[features_scaling2])
X2_trainval.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
37,0.0694,0.0465,0.0515,0.0601,0.0633,0.0827,0.0886,0.0671,0.0888,0.0860,0.0000,0.0000,0.0541,0.1146,0.1388,0.1571,0.1459,0.1752,0.0733,0.0474,0.2146


Aprontar também o test, já aplicando a reescala.

In [559]:
X2_test[features_scaling2] = scaler2_tv.transform(X2_test[features_scaling2])
X2_test.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
5,0.0604,0.0285,0.0302,0.0408,0.0452,0.0693,0.0763,0.0721,0.1076,0.0731,0.0216,0.1067,0.1885,0.1564,0.0992,0.0937,0.0613,0.0713,0.1791,0.1600,0.4081


### Encoding

Encodar apenas as features selecionadas do treino com OrdinalEncoder.

In [562]:
encoder2 = OrdinalEncoder(categories=[['Baixo', 'Médio', 'Alto']], dtype=int)
y2_train = encoder2.fit_transform(y2_train.values.reshape(-1,1))
encoder2.categories_

[array(['Baixo', 'Médio', 'Alto'], dtype=object)]

Aplica o encoder já treinado apenas na feature selecionada de validação.

In [563]:
y2_val = encoder2.transform(y2_val.values.reshape(-1,1))
y2_val[:3]

array([[2],
       [0],
       [1]])

Já deixar pronto também trainval, para avaliar a capacidade de generalização no final.

In [564]:
encoder2_tv = OrdinalEncoder(categories=[['Baixo', 'Médio', 'Alto']], dtype=int)
y2_trainval = encoder2_tv.fit_transform(y2_trainval.values.reshape(-1,1))
y2_trainval[:3]

array([[1],
       [2],
       [1]])

Aprontar também o test, já aplicando as transformações.

In [565]:
y2_test = encoder2_tv.transform(y2_test.values.reshape(-1,1))
y2_test[:3]

array([[2],
       [2],
       [0]])

## Seleção de Features

Será utilizada a técnica de feature selection baseada em algoritmo (Random Forest), mas desta vez com um RandomForestClassifier.

Ela foi a melhor na análise 1, e pela experiência, apresenta boa robustez de resultados.

In [566]:
sel_mbfs = SelectFromModel( RandomForestClassifier(n_estimators=10, random_state=0), threshold="0.6*median") 
sel_mbfs.fit(X2_train, y2_train.ravel()).transform(X2_train)
features_mbfs = sel_mbfs.get_feature_names_out()
features_mbfs

array(['populacao', 'pop_de20a24', 'pop_de25a34', 'pop_de50a59',
       'pop_mais_de60', 'pop_alvo', 'domicilios_a1', 'domicilios_a2',
       'domicilios_b1', 'domicilios_b2', 'domicilios_alvo', 'renda_media',
       'faturamento'], dtype=object)

Mantendo apenas as features selecionadas nos demais datasets.

In [567]:
X2_train_mbfs = X2_train[features_mbfs].copy() 
X2_val_mbfs = X2_val[features_mbfs].copy() 
X2_trainval_mbfs = X2_trainval[features_mbfs].copy() 
X2_test_mbfs = X2_test[features_mbfs].copy() 
X2_train_mbfs.head(1)

,populacao,pop_de20a24,pop_de25a34,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_alvo,renda_media,faturamento
18,0.0446,0.0394,0.0463,0.0463,0.0404,0.0467,0.0000,0.0077,0.0244,0.0680,0.0305,0.0804,0.3509


## Implementação de Baseline

Nesta análise, também será implementado o KNN como modelo de base. 

Se tratando de um problema de classificação, será utilizado o KNN classifier da biblioteca scikit-learn.

Manter apenas features numéricas, respeitando a premissa do KNN.

In [568]:
X2_train = X2_train.select_dtypes(include=['int64', 'float64'])
X2_val = X2_val.select_dtypes(include=['int64', 'float64'])
X2_train.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
18,0.0446,0.0375,0.0361,0.0380,0.0394,0.0463,0.0470,0.0463,0.0404,0.0467,0.0000,0.0077,0.0244,0.0680,0.0475,0.0316,0.0263,0.0501,0.0305,0.0804,0.3509


Instanciar um KNN Classifier, treinar com os dados de treino, e realizar predições com os dados de validação.

Será parametrizado neste ciclo um número de k=3, visando um modelo mais simples que o K=5 default, com a expectativa de uma maior capacidade de generalização. Nos próximos ciclos, serão avaliados diferentes valores de K, para comparação.

In [51]:
knn_cla = KNeighborsClassifier(n_neighbors=3)
knn_cla.fit(X2_train, y2_train)
knn_cla_yhat = knn_cla.predict(X2_val)

In [52]:
knn_cla_yhat[:3]

array(['Médio', 'Baixo', 'Médio'], dtype=object)

A métrica utilizada para medir a performance do modelo será a acurácia média.
- Varia entre 0 e 100%, e mede quantas predições o modelo acertou, dentre todas as previsões realizadas.
- Busca-se portanto, o valor mais próximo de 100% possível, ou seja, a previsão de potencial mais próxima da realidade possível.

In [53]:
print("Acurácia com dados de Validação: {:.4f}".format(knn_cla.score(X2_val, y2_val)))

Acurácia com dados de Validação: 0.5714


Pode-se calcular também a acurácia média a partir da predição do modelo, através do pacote "metrics" do scikit-sklearn.

In [54]:
print(f"Acurácia com dados de Validação: {round(accuracy_score(y2_val, knn_cla_yhat),4)}")

Acurácia com dados de Validação: 0.5714


A acurácia do KNN foi satisfatória neste primeiro ciclo. Possivlmente a métrica pode ser melhorada, por meio de:
- Reescala de dados.
- Seleção de features, removendo as não informativas e aquelas não relacionadas ao público alvo.
- Seleção de diferentes valores para K, bem como tunagem de hiperparâmetros no modelo.
- Avaliação da presença de outliers, que podem distorcer os cálculos de distâncias.

Nos próximos ciclos, estes temas serão tratados, e novos algoritmos com diferentes abordagens também serão implementados.

No último ciclo, será avaliada a capacidade de generalização do modelo final (de melhor performance), realizando a previsão com os dados de teste.

## Comparação com o Baseline

Esta seção e as subsequêntes dentro da análise 2, foram implementadas no ciclo 3.

Percebe-se que 3 dos 4 pontos de melhoria identificados no ciclo 1 (seção anterior) foram sanados. O quarto (seleção de diferentes números de K) será também tratado na sequência.

Mapeamento dos split do dataset já preparados, para facilitar o treinamento de modelos:
- X --> X2_train_mbfs, X2_val_mbfs, X2_trainval_mbfs, X2_test_mbfs
- y --> y2_train,      y2_val,      y2_trainval,      y2_test

Treinar KNN com mesmos parâmetros do ciclo 1.

In [573]:
knn_cla_c2 = KNeighborsClassifier(n_neighbors=3)
knn_cla_c2.fit(X2_train_mbfs, y2_train.ravel())
knn_cla_c2_yhat = knn_cla_c2.predict(X2_val_mbfs)

Inspeção manual de Amostras.

In [574]:
print(f'Previsto: {knn_cla_c2_yhat[:20]}')
print(f'Real:     {y2_val[:20].ravel()}')

Previsto: [2 0 1 2 2 2 0 0 0 0 2 0 1 0 2 0 2 2 1 2]
Real:     [2 0 1 2 2 2 1 1 0 0 1 0 0 0 1 0 2 2 1 2]


Comparação de performance com o ciclo 1.

In [584]:
print(f"Acurácia em Validação: {round(accuracy_score(y2_val, knn_cla_c2_yhat),4)}")

Acurácia em Validação: 0.75


A acurácia teve um aumento neste ciclo 2 em relação ao primeiro, como pode ser visto acima.

No ciclo 1, foi utilizada a acurácia média, pelo desbalanceamento entre as classes ser pequeno. 

Ocorre que como a variável alvo possui muito poucas amostras, este pequeno desbalanceamento tem um alto impacto.
- Isto, porque a acurácia atribui o mesmo peso para qualquer classe. Portanto, o valor da acurácia para problemas com classes desbalanceadas é dominado pelo acerto da classe majoritária.

A métrica que seguirá sendo utilizada a partir daqui é o F1 Score. 
- F1-Score: Resume precision e recall, sendo a média harmônica entre ambos.
- É usado como métrica em datasets desbalanceados de classificação.

In [583]:
print(f"F1-Score em Validação: {round(f1_score( y2_val, knn_cla_c2_yhat, average='weighted' ),4)}")

F1-Score em Validação: 0.7292


Nas próximas seções, serão utilizadas técnicas para melhoria de performance dos modelos, bem como outros modelos também serão experimentados.

## KNN Classifier

A seguir, será implmentado um KNN Classifier com Grid Search + Cross Validation.

In [585]:
knn_param_grid2 = {'n_neighbors':[1,2,3,4,5,6,7,8], 'weights':['uniform','distance']}
#instanciar GridSearchCV com KNNRegressor e treinar o modelo
knn_cla_gscv = GridSearchCV(KNeighborsClassifier(), knn_param_grid2, cv=10).fit(X2_train_mbfs, y2_train.ravel())

Analisar os melhores parâmetros do KNN. 

In [586]:
print(f"Melhores parâmetros: {knn_cla_gscv.best_params_}")

Melhores parâmetros: {'n_neighbors': 7, 'weights': 'uniform'}


Predição em cima dos dados de validação.

In [587]:
knn_cla_gscv_yhat = knn_cla_gscv.predict(X2_val_mbfs)

Performance.

In [589]:
knn_cla_gscv_score = f1_score(y2_val, knn_cla_gscv_yhat, average='weighted')
print(f"KNN com Cross Val e Grid Search - F1-Score em Validação: {round(knn_cla_gscv_score,4)}")

KNN com Cross Val e Grid Search - F1-Score em Validação: 0.6618


O resultado do KNN com seus melhores parâmetros não aumentou, mas servirá de base, para comparação com os próximos modelos implementados.

Criar dataframe para comparação de resultados, e atribuir o do KNN.

In [619]:
df_an2_scores = pd.DataFrame(columns=['model_name','mean_accuracy_score'])
df_an2_scores.loc[len(df_an2_scores)] = ['knn_cla_gscv', knn_cla_gscv_score]
df_an2_scores

,model_name,mean_accuracy_score
0,knn_cla_gscv,0.6618


## Logistic Regression

A regressão logística é algoritmo linear de classificação, que encontraa as relações entre dois fatores de dados, e usa essa relação para prever o valor de um fator com base no outro.
- O parâmetro que determina a força da regularização é chamado 'C'. Quanto menor o 'C' mais simples será o modelo. Logo, mais regularização tende a ter (menor flexibilidade), tendendo mais para a generalização.

In [591]:
lreg_param_grid2 = {'C': [0.01, 0.1, 1, 10, 100]}
#instanciar GridSearchCV com LogisticRegression e treinar o modelo
lreg_cla_gscv = GridSearchCV(LogisticRegression(max_iter=1000), lreg_param_grid2, cv=10).fit(X2_train_mbfs, y2_train.ravel())

Analisar os melhores parâmetros da Reg Log. 

In [592]:
print(f"Melhores parâmetros: {lreg_cla_gscv.best_params_}")

Melhores parâmetros: {'C': 100}


Predição em cima dos dados de validação.

In [593]:
lreg_cla_gscv_yhat = lreg_cla_gscv.predict(X2_val_mbfs)

Performance.

In [595]:
lreg_cla_gscv_score = f1_score(y2_val, knn_cla_gscv_yhat, average='weighted')
print(f"Logistic Regression com Cross Val e Grid Search - F1-Score em Validação: {round(lreg_cla_gscv_score,4)}")

Logistic Regression com Cross Val e Grid Search - F1-Score em Validação: 0.6618


O resultado foi o mesmo que o obtido com o KNN. Isto não é comum, mas possivelmente ocorreu pelo fato de y2_val possuir poucos registros.

Inserir no dataframe de resultados.

In [620]:
df_an2_scores.loc[len(df_an2_scores)] = ['lreg_cla_gscv', lreg_cla_gscv_score]

## XGBoost Classifier

In [606]:
xgb_param_grid = {'n_estimators':[50, 100], 'max_depth':[4, 6], 'min_child_weight':[4,9], 'colsample_bytree':[0.5], 'eta':[0.1,0,2]}
#instanciar GridSearchCV com XGBClassifier e treinar o modelo
xgb_cla_gscv = GridSearchCV(XGBClassifier(), xgb_param_grid, cv=10).fit(X2_train_mbfs, y2_train.ravel())

Analisar os melhores parâmetros do XGBoost. 

In [607]:
print(f"Melhores parâmetros: {xgb_cla_gscv.best_params_}")

Melhores parâmetros: {'colsample_bytree': 0.5, 'eta': 0.1, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 50}


Predição em cima dos dados de validação.

In [608]:
xgb_cla_gscv_yhat = xgb_cla_gscv.predict(X2_val_mbfs)

Performance.

In [609]:
xgb_cla_gscv_score = f1_score(y2_val, xgb_cla_gscv_yhat, average='weighted')
print(f"XGBClassifier com Cross Val e Grid Search - F1-Score em Validação: {round(xgb_cla_gscv_score,4)}")

XGBClassifier com Cross Val e Grid Search - F1-Score em Validação: 0.7671


O resultado do XGBoost foi mais interessante que a dos modelos anteriores.

Inserir no dataframe de resultados.

In [621]:
df_an2_scores.loc[len(df_an2_scores)] = ['xgb_cla_gscv', xgb_cla_gscv_score]

## Performance dos Modelos

A tabela abaixo detalha a melhor performance obtida com cada modelo de machine learning, já com fine tuning e cross validation.

In [622]:
df_an2_scores

,model_name,mean_accuracy_score
0,knn_cla_gscv,0.6618
1,lreg_cla_gscv,0.6618
2,xgb_cla_gscv,0.7671


In [ ]:
O modelo de melhor performance foi o XGBoost, e ele será utilizado para a sequência da análise.

## Performance de Generalização

In [623]:
xgb_tuning = {'colsample_bytree': 0.5, 'eta': 0.1, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 50}
#treina com melhores parâmetros
xgb_cla_gscv_tuned = xgb.XGBClassifier( 
                                        max_depth = xgb_tuning['max_depth'],
                                        min_child_weight = xgb_tuning['min_child_weight'],
                                        n_estimators = xgb_tuning['n_estimators'],
                                        colsample_bytree = xgb_tuning['colsample_bytree'],
                                        eta = xgb_tuning['eta']
                                    ).fit( X2_train_mbfs, y2_train.ravel() )
#prediz contra teste
xgb_cla_gscv_tuned_yhat = xgb_cla_gscv_tuned.predict( X2_test_mbfs )

In [626]:
xgb_cla_gscv_tuned_score = f1_score(y2_test, xgb_cla_gscv_tuned_yhat, average='weighted')
print(f"XGBClassifier com Cross Val e Grid Search - F1-Score em Teste: {round(xgb_cla_gscv_tuned_score,4)}")

XGBClassifier com Cross Val e Grid Search - F1-Score em Teste: 0.8744


A capacidade de generalização foi bastante satisfatória, consideravelmente acima da performance em validação.

Na próxima seção, o modelo final será treinado.

## Modelo Final

Um modelo final será agora treinado utilizando os dados de X2_trainval (treino + validação), com o melhor set de parâmetros. 

Busca-se com isto aumentar ainda mais a performance do modelo na previsão de dados de produção.

Carregar dados de SP (produção).

In [648]:
df_sp_raw2 = pd.read_csv('../data/interim/df_sp_an1_done.csv', index_col=0)
df_sp2 = df_sp_raw2.copy()
print(df_sp2.shape)
df_sp2.head(1)

(296, 24)


,codigo,bairro,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
160,355030251,A. E. Carvalho,São Paulo,SP,94034,12668,6853,9836,7487,14535,21549,10598,10508,0,253,2197,4368,6681,7011,2247,5670,1501,209502.5000,NaN


Aplicar as mesmas adequações nos dados do df_rj da análise 2 para df_sp.

Derivar features.

In [649]:
df_sp2["pop_alvo"] = df_sp2["pop_de25a34"] + df_sp2["pop_de35a49"]
df_sp2["domicilios_alvo"] = df_sp2["domicilios_a1"] + df_sp2["domicilios_a2"] + df_sp2["domicilios_b1"] + df_sp2["domicilios_b2"]

Filtrar features.

In [650]:
df_sp2 = df_sp2[features_scaling2].copy()
df_sp2.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
160,94034,12668,6853,9836,7487,14535,21549,10598,10508,36084,0,253,2197,4368,6681,7011,2247,5670,6818,1501,209502.5000


Preencher com zero os 3 registros com '-' em renda_media.

In [656]:
df_sp2.loc[~df_sp2.renda_media.str.isnumeric(), 'renda_media'] = 0

Aplicar preperação nos dados com scaler já pronto.

In [657]:
df_sp2[features_scaling2] = scaler2.transform(df_sp2)
df_sp2.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento
160,0.3757,0.3874,0.4292,0.4695,0.3711,0.3846,0.4055,0.3390,0.2041,0.3968,0.0000,0.0274,0.1135,0.3291,0.2741,0.3122,0.4923,0.5103,0.1436,0.0498,0.0783


Manter mesmas features de X2_trainval_mbfs

In [662]:
df_sp2.columns

Index(['populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24',
       'pop_de25a34', 'pop_de35a49', 'pop_de50a59', 'pop_mais_de60',
       'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1',
       'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d',
       'domicilios_e', 'domicilios_alvo', 'renda_media', 'faturamento'],
      dtype='object')

In [663]:
X2_trainval_mbfs.columns

Index(['populacao', 'pop_de20a24', 'pop_de25a34', 'pop_de50a59',
       'pop_mais_de60', 'pop_alvo', 'domicilios_a1', 'domicilios_a2',
       'domicilios_b1', 'domicilios_b2', 'domicilios_alvo', 'renda_media',
       'faturamento'],
      dtype='object')

In [664]:
df_sp2 = df_sp2[['populacao', 'pop_de20a24', 'pop_de25a34', 'pop_de50a59', 'pop_mais_de60', 'pop_alvo',
'domicilios_a1','domicilios_a2', 'domicilios_b1', 'domicilios_b2', 'domicilios_alvo', 'renda_media', 'faturamento']]
df_sp2.head(1)

,populacao,pop_de20a24,pop_de25a34,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_alvo,renda_media,faturamento
160,0.3757,0.3711,0.3846,0.3390,0.2041,0.3968,0.0000,0.0274,0.1135,0.3291,0.1436,0.0498,0.0783


In [666]:
X2_trainval_mbfs.head(1)

,populacao,pop_de20a24,pop_de25a34,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_alvo,renda_media,faturamento
37,0.0694,0.0633,0.0827,0.0671,0.0888,0.0860,0.0000,0.0000,0.0541,0.1146,0.0733,0.0474,0.2146


Treinamento de modelo final com o melhor set de parâmetros, utilizando os dados treino + validação.

Ele fará a previsão dos dados de produção (df_sp).

In [674]:
xgb_tuning = {'colsample_bytree': 0.5, 'eta': 0.1, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 50}
#treina com melhores parâmetros
xgb_cla_gscv_tuned_prod = xgb.XGBClassifier( 
                                        max_depth = xgb_tuning['max_depth'],
                                        min_child_weight = xgb_tuning['min_child_weight'],
                                        n_estimators = xgb_tuning['n_estimators'],
                                        colsample_bytree = xgb_tuning['colsample_bytree'],
                                        eta = xgb_tuning['eta']
                                    ).fit( X2_train_mbfs, y2_train.ravel() )
#prediz com dados de teste (df_sp)
xgb_cla_gscv_production_yhat = xgb_cla_gscv_tuned_prod.predict( df_sp2 )

Unificar a previsão com o dataset SP original. 

In [709]:
df_sp_raw2['potencial'] = xgb_cla_gscv_production_yhat
df_sp_raw2.potencial[10:13]

170    1
171    2
172    0
Name: potencial, dtype: int64

Trazer de volta a legenda da variável alvo.

In [711]:
df_sp_raw2.potencial = encoder2.inverse_transform(df_sp_raw2.potencial.values.reshape(-1, 1))
df_sp_raw2.potencial[10:13]

170    Médio
171     Alto
172    Baixo
Name: potencial, dtype: object

O dataset SP com as com as 296 previsões de potencial por bairro será exportado, para ser complementado com a análise 3 (clusterização).

In [712]:
print(df_sp_raw2.shape)
df_sp_raw2.head(3)

(296, 24)


,codigo,bairro,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
160,355030251,A. E. Carvalho,São Paulo,SP,94034,12668,6853,9836,7487,14535,21549,10598,10508,0,253,2197,4368,6681,7011,2247,5670,1501,209502.5000,Alto
161,35503020,Aclimação,São Paulo,SP,32791,2297,1017,2096,2197,5341,7281,4917,7645,1413,1734,3704,2351,1946,827,291,1617,5920,1508728.4000,Alto
162,355030285,Adventista,São Paulo,SP,104193,15070,7343,10631,8657,17749,23364,11567,9812,0,0,1423,4875,8595,10082,3111,5776,1284,165232.6900,Médio


In [713]:
df_sp_raw2.to_csv('../data/interim/df_sp_an1e2_done.csv')

# 4 Lições aprendidas

Lições aprendidas até o ciclo 4:

- Em projetos com mais de um problema de negócio (caso deste, com 3 análises), desenvolver notebooks distintospara cada análise, visando maior organização.
- Tratar as features de produção (df_sp neste caso) conforme for desenvolvendo o projeto, pra dispensando tratativas na hora de criar o modelo final. 
- Caso não utilize pipelines, a seguinte ordem dos passos (para cada análise) desde o início facilita o desenvolvimento: 1) Split de dados (antes da modelagem, para evitar data leakeage), 2) Modelagem (fit_transform no treino, transforme em val, test e dado prod) 3) Feature Selection (deve ser após o split, pois ao utilizar modelos, requerem train e val).
    
PS: Discutir os tópicos com outros cientistas de dados, para obter suas perspectivas baseadas na sua experiência.